In [1]:
import lionagi as li

In [2]:
class Coder(li.Worker):
    name = "coder"
    
    @li.work(capacity=5, timeout=5, refresh_time=1)
    async def chat(self, instruction=None, context=None):
        branch = li.Branch()
        return await branch.chat(instruction=instruction, context=context)

In [3]:
coder = Coder()

In [4]:
# calling the function only adds it to the internal worklog
instruction = "write a serious hello world according to the given language, answer only in emoji"
languages = ["python", "c++", "java", "javascript", "ruby", "php", "c#", "matlab"]

for i in languages:
    await coder.chat(instruction=instruction, context=i)

In [5]:
chat_func_log = coder.work_functions['chat'].worklog

In [6]:
print("Total number of work items in the worklog: \t\t", len(chat_func_log.pile))
print("Total number of pending items in the worklog: \t\t", len(chat_func_log.pending))
print("Available capacity remaining in the async queue: \t", chat_func_log.queue.available_capacity)

Total number of work items in the worklog: 		 8
Total number of pending items in the worklog: 		 8
Available capacity remaining in the async queue: 	 5


In [7]:
await chat_func_log.forward()
await chat_func_log.queue.process()

In [8]:
for language, work in zip(languages, chat_func_log.completed_work.values()):
    print(f"{language}: {work.result}")

python: 🐍✨🌍👋
c++: 👋🌍👨‍💻📝💻
java: ☕️🌍👋
javascript: 🌐✨🖥️👋🌍
ruby: 🔴💎🌍


In [10]:
await chat_func_log.forward()
await chat_func_log.queue.process()

In [11]:
for language, work in zip(languages, chat_func_log.completed_work.values()):
    print(f"{language}: {work.result}")

python: 🐍✨🌍👋
c++: 👋🌍👨‍💻📝💻
java: ☕️🌍👋
javascript: 🌐✨🖥️👋🌍
ruby: 🔴💎🌍
php: 👋🌍✉️🐘💻
c#: 👋🌍
matlab: 👋🌍🔢


In [14]:
chat_func_log.pile.to_df()

,ln_id,created,metadata,content,status,result,error,completion_timestamp,duration
0,03747bddb08d5639015595c6781c0445,2024-05-10T14:59:51.467974,"{'last_updated': {'status': (2, '2024-05-10T14...",None,WorkStatus.COMPLETED,🐍✨🌍👋,None,2024-05-10T14:59:53.166196,1.697971
1,f64d3772114ec7190ecca8535c15c022,2024-05-10T14:59:51.468021,"{'last_updated': {'status': (2, '2024-05-10T14...",None,WorkStatus.COMPLETED,👋🌍👨‍💻📝💻,None,2024-05-10T14:59:53.356393,1.888120
2,821897cb62c88ae235017c8ed2c13e2c,2024-05-10T14:59:51.468040,"{'last_updated': {'status': (2, '2024-05-10T14...",None,WorkStatus.COMPLETED,☕️🌍👋,None,2024-05-10T14:59:53.356597,1.771237
3,ef5bef14ca68bd66d7685a846105b891,2024-05-10T14:59:51.468056,"{'last_updated': {'status': (2, '2024-05-10T14...",None,WorkStatus.COMPLETED,🌐✨🖥️👋🌍,None,2024-05-10T14:59:53.356824,1.666174
4,3e946482d337e3a0909ca4b087a3fa77,2024-05-10T14:59:51.468069,"{'last_updated': {'status': (2, '2024-05-10T14...",None,WorkStatus.COMPLETED,🔴💎🌍,None,2024-05-10T14:59:53.357062,1.468743
5,6a57b6be98ee986d849816694549f758,2024-05-10T14:59:51.468083,"{'last_updated': {'status': (2, '2024-05-10T14...",None,WorkStatus.COMPLETED,👋🌍✉️🐘💻,None,2024-05-10T14:59:53.376120,1.877940
6,58d1f0bc2e0d671d855d6851070c432e,2024-05-10T14:59:51.468097,"{'last_updated': {'status': (2, '2024-05-10T14...",None,WorkStatus.COMPLETED,👋🌍,None,2024-05-10T14:59:53.376727,1.015103
7,25903d8ac1d651945a69095c9849a3eb,2024-05-10T14:59:51.468113,"{'last_updated': {'status': (2, '2024-05-10T14...",None,WorkStatus.COMPLETED,👋🌍🔢,None,2024-05-10T14:59:53.376946,1.148245
